In [ ]:
!git clone https://github.com/kms9561/Inflation_Forecast.git

In [1]:
import pickle

import pandas as pd
from pandas.tseries.offsets import QuarterEnd, QuarterBegin, MonthEnd, MonthBegin, BDay
from pandas.tseries.offsets import DateOffset

import requests
import math
import joblib

import numpy as np
from numpy import asarray

from datetime import datetime
from multiprocessing import Process

import glob
import os

from time import time
from tqdm import tqdm
import itertools

idx = pd.IndexSlice

In [3]:
def get_bidas_data(req_ids, alias_nm, 
                     start_d=None, end_d=None,
                     period_trim=False):
    # API 호출
    API = "http://datahub.boknet.intra/api/v1/obs/lists"
    res = requests.post(API, data={"ids":req_ids})
    data_list = res.json()["data"][0]
    
    # API 호출로 받은 결과를 Data Frame으로 저장
    data = pd.DataFrame()
    for alias, value in zip(alias_nm, data_list):
        try:
            df = pd.DataFrame(value["observations"])
            df.set_index("period", inplace=True)
            df.index = pd.to_datetime(df.index)
            df.columns = [alias]
            data = df.copy() if not len(data) else data.join(df, how="outer")
        except:
            print(f'{alias} is not imported.')
    
    # 옵션에 따라 시작일, 종료일, Trim 적용
    if start_d:
        data = data[data.index >= start_d]
    if end_d:
        data = data[data.index <= end_d]
    if period_trim:
        data.index = data.index.to_period('M')
    return data

In [3]:
def get_bidas_data_2018(group):
    if group == '':
        data=pd.DataFrame()
        df=pd.read_excel('/content/Inflation_Forecast/input/data_bidas.xlsx')
        df.set_index("period", inplace=True)
        data=df.copy()
        return data
    data=pd.DataFrame()
    start_d='2018-01-01'
    end_d='2018-12-31'
    df=pd.read_excel('/content/Inflation_Forecast/input/BIDAS DATA_'+group+'.xlsx')
    df.set_index("period", inplace=True)
    df.index = pd.to_datetime(df.index)
    data = df.copy() if not len(data) else data.join(df, how="outer")
    data = data[data.index >= start_d]
    data = data[data.index <= end_d]
    return data

In [25]:
# 관리물가, 관리제외물가지수 생성 함수
def gen_P_adm_eadm_v2(df, w, b_date, e_date):

    df1 = df.iloc[:,:35]
    df2 = df.iloc[:,35:]
    
    # 관리물가지수 생성
    w1 = w.iloc[0:35] / w.iloc[0:35].sum()
    w1 = asarray(w1)
    P_a = df1.dot(w1)
    c_name = 'P_adm'
    P_a = pd.DataFrame(P_a, columns=[c_name])
    #P_a = (np.log(P_a) - np.log(P_a.shift(12)))*100 # 데이터 변환
                                        
    # 관리제외물가지수 생성
    w2 = w.iloc[35:] / w.iloc[35:].sum()
    w2 = asarray(w2)


    
    P_ea = df2.dot(w2)
    c_name = 'P_adm'
    P_ea = pd.DataFrame(P_ea, columns=[c_name])
    #P_ea = (np.log(P_ea) - np.log(P_ea.shift(12)))*100 # 데이터 변환
                                        
    return P_a, P_ea

In [7]:
b_date = '2000-1'

c_date = datetime.now()
c_year = c_date.year
c_month = c_date.month
e_date = f'{c_year}-{c_month}'
b_date='2018-01-01'
e_date='2018-12-31'

In [9]:
s3_repo_path = "/content/Inflation_Forecast/input"

In [11]:
vspec = pd.read_excel(f'{s3_repo_path}/data_list_all_v8.xlsx', index_col = None)
vspec.index.names = [None]

vspec_import = vspec.loc[vspec['Data ID'] != 'calculated']
vspec

,Group,Series Name,cat,Acode,Tcode1,Adcode,Gcode,My ID,Data ID
0,소비자물가,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,1.0,P_cpi_1,KOSIS-101_DT_1J22001-M-T-T10-0-14101_ausgabe
1,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_2,KOSIS-101_DT_1J22001-M-T-T10-A-14101_ausgabe
2,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_3,KOSIS-101_DT_1J22001-M-T-T10-C-14101_ausgabe
3,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_4,KOSIS-101_DT_1J22001-M-T-T10-D-14101_ausgabe
4,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_5,KOSIS-101_DT_1J22001-M-T-T10-E-14101_ausgabe
...,...,...,...,...,...,...,...,...,...
317,NaN,통합재정수지 (월 1999.12~2023.05)-월-재정수지-자본지출-십억원,3,w3lag2,2,0.0,3.0,F_fb_5,KOSIS-102_DT_102N_AD01-M-16102AD1-15102AD123-1...
318,NaN,통합재정수지 (월 1999.12~2023.05)-월-재정수지-통합재정수지-십억원,1,w3lag2,3,0.0,3.0,F_fb_6,KOSIS-102_DT_102N_AD01-M-16102AD1-15102AD130-1...
319,텍스트,뉴스심리지수(실험적 통계)-월-뉴스심리지수,1,w1lag1,1,1.0,2.0,T_nsi,NECOS-521U001-M-A001
320,NaN,US Economic Policy Uncertainty Composite Index,1,w2lag1,1,1.0,1.0,T_usepu,BBG-BBG-EPUCCUSM_Index


In [12]:
today = datetime.today()
vintage_date = today.strftime('%Y-%m-%d')

In [15]:
groups = ['P', 'PB', 'L', 'C', 'LM', 'IE', 'EI', 'RE', 'EG', 'S', 'GB', 'F', 'T']

In [17]:
adm_vspec = pd.read_excel(f'{s3_repo_path}/AdmPrices.xlsx', sheet_name='AdmPrices_list', index_col = None)
adm_vspec.index.names = [None]
adm_var_list = [var for var in adm_vspec['My ID']]
#dfa = get_bidas_data(adm_vspec.loc[adm_vspec['My ID'].isin(adm_var_list), 'Data ID'], adm_var_list).loc[b_date:e_date].astype(float).resample('M').mean()
dfa = get_bidas_data_2018('').loc[b_date:e_date]


#여기까지 조치함

In [45]:
DF = pd.DataFrame()
DF_raw = pd.DataFrame()

for group in groups:
    
    var_list = [var for var in vspec_import['My ID'] if var.split('_')[0] == group]
    #df = get_bidas_data(vspec.loc[vspec['My ID'].isin(var_list), 'Data ID'], var_list).loc[b_date:e_date].astype(float).resample('M').mean()
    df = get_bidas_data_2018(group)
    DF_raw = pd.concat([DF_raw, df], axis=1)
    if group == 'IE':
        df['IE_mir_8'] = df['IE_mir_5'] - df['IE_mir_2']
        
    elif group == 'P':
        adm_vspec = pd.read_excel(f'{s3_repo_path}/AdmPrices.xlsx', sheet_name='AdmPrices_list', index_col = None)
        adm_vspec.index.names = [None]
        adm_var_list = [var for var in adm_vspec['My ID']]
        #dfa = get_bidas_data(adm_vspec.loc[adm_vspec['My ID'].isin(adm_var_list), 'Data ID'], adm_var_list).loc[b_date:e_date].astype(float).resample('M').mean()
        dfa = get_bidas_data_2018('').loc[b_date:e_date]
        P_a, P_ea = gen_P_adm_eadm_v2(dfa, adm_vspec.weights, b_date, e_date)

        
        df['P_adm'] = P_a
        df['P_eadm'] = P_ea
        
    elif group == 'L':
        df.loc[:, 'L_wh_1'] = df.loc[:, ['L_wh_1_a', 'L_wh_1_b', 'L_wh_1_c']].mean(axis=1)
        df.loc[:, 'L_wh_2'] = df.loc[:, ['L_wh_2_a', 'L_wh_2_b', 'L_wh_2_c']].mean(axis=1)
        df.loc[:, 'L_wh_3'] = df.loc[:, ['L_wh_3_a', 'L_wh_3_b', 'L_wh_3_c']].mean(axis=1)
        df.loc[:, 'L_tw_1'] = df.loc[:, ['L_tw_1_a', 'L_tw_1_b', 'L_tw_1_c']].mean(axis=1)
        df.loc[:, 'L_tw_2'] = df.loc[:, ['L_tw_2_a', 'L_tw_2_b', 'L_tw_2_c']].mean(axis=1)
        df.loc[:, 'L_tw_3'] = df.loc[:, ['L_tw_3_a', 'L_tw_3_b', 'L_tw_3_c']].mean(axis=1)
        df.loc[:, 'L_vjp'] = df.loc[:, ['L_vjp_a', 'L_vjp_b']].mean(axis=1)
        df['L_vu'] = df['L_vjp'] / df['L_eap_4'] # 실업자대비 빈일자리 비율

        # drop columns L_wh_1_a, L_wh_1_b, ..., L_tw_3_c
        for p in ['L_' + p + '_' + q + '_' + r for p in ['wh', 'tw'] for q in ['1', '2', '3'] for r in ['a', 'b', 'c']]:
            df = df.drop(p, axis=1)
        df = df.drop(['L_vjp_a', 'L_vjp_b'], axis=1)            
        
    elif group == 'RE':
        df.loc[:, 'RE_ccs_1'] = df.loc[:, ['RE_ccs_1_a', 'RE_ccs_1_b']].mean(axis=1)
        df.loc[:, 'RE_ccs_2'] = df.loc[:, ['RE_ccs_2_a', 'RE_ccs_2_b']].mean(axis=1)

        # drop columns RE_ccs_2_a ...
        df = df.drop(['RE_ccs_1_a', 'RE_ccs_1_b', 'RE_ccs_2_a', 'RE_ccs_2_b'], axis=1)  
        
    else:
        pass

    for var in df.columns:
        
        dfi = df[var]
        if vspec.loc[vspec['My ID'].eq(var), 'Tcode1'].values[0] == 2:
        
            if (dfi <= 0).any().any():
                dfi = dfi.pct_change(12)*100
            elif var == 'P_cpi_1': # Nan 인 경우 ffill 되어 마지막 값 nan인 경우 이상한 값이 채워짐
                dfi = dfi.dropna().pct_change(12)*100
            else:
                dfi = (np.log(dfi) - np.log(dfi.shift(12)))*100
                
        elif vspec.loc[vspec['My ID'].eq(var), 'Tcode1'].values[0] == 3:
            dfi = dfi - dfi.shift(1)
            
        DF = pd.concat([DF, dfi], axis=1)

DF.index = pd.to_datetime(DF.index)

DF_raw.index = pd.to_datetime(DF_raw.index)

## P_ppi_2, 생산자물가지수(기본분류)-상품 2020년으로 기준년 개편되면서 삭제됨

In [49]:
#s3_repo_path = f"s3://newtech/public/inf_nowcasting/input/mdata/{vintage_date}.xlsx"

#DF.to_excel(s3_repo_path, sheet_name="data")
DF.to_csv(f"{s3_repo_path}/mdata/{vintage_date}.csv")
DF_raw.to_csv(f"{s3_repo_path}/mdata/{vintage_date}_raw.csv")